# Práctica de planificación bajo incertidumbre

### Inteligencia Artificial
### Grado en Ingeniería Informática - Ingeniería del Software
### Universidad de Sevilla

El paquete de _Python_ [pymdptoolbox](https://github.com/sawcordwell/pymdptoolbox) proporciona un marco de trabajo para procesos de decisión de Markov.

El paquete proporciona tres módulos:
* El módulo _mdp_ es el que implementa los procesos de decisión de Markov y los algoritmos de iteración de valores y de políticas, entre otros.
* El módulo _util_ proporciona algunas funciones para comprobar la correcta descripción del proceso de decisión de Markov, como por ejemplo que la función de transición de cada acción aplicada a cada estado es una distribución de probabilidad (las probabilidades de los nuevos estados están en el intervalo $[0, 1]$ y suman $1$).
* El modulo _example_ proporciona algunos ejemplos.

En esta práctica solo usaremos el primero de esos módulos.

In [99]:
import mdptoolbox.mdp as mdp

Los algoritmos para calcular una política óptima utilizan operaciones matriciales, por lo que también necesitaremos hacer uso del paquete [Numpy](https://numpy.org/).

In [173]:
import numpy

# Ejemplo del robot y las cinco localizaciones

Recordemos que en el tema hemos visto un ejemplo de proceso de decisión de Markov en el que tenemos un robot que puede encontrarse en una de entre cinco localizaciones y trata de moverse entre ellas, con el objetivo de llegar a una localización concreta.

Los estados del sistema están representados por un predicado `en`, que indica la localización en la que se encuentra el robot.

In [188]:
#estados = ['en(l1)', 'en(l2)', 'en(l3)', 'en(l4)', 'en(l5)']
estados = ['M','PC2','PC3','PC4','PC5','A2', 'A3', 'A4', 'A5'] 


Las posibles acciones que puede realizar el robot son `esperar` o `ir` de una localización a otra (pero no para cualquier par de localizaciones hay un camino directo).

In [189]:
'''
acciones = ['esperar',
            'ir(l1, l2)', 'ir(l1, l4)',
            'ir(l2, l1)', 'ir(l2, l3)',
            'ir(l3, l2)', 'ir(l3, l4)',
            'ir(l4, l1)', 'ir(l4, l3)', 'ir(l4, l5)',
            'ir(l5, l2)', 'ir(l5, l4)']
'''
acciones = ['actua','viaja']


En realidad, las listas de estados y acciones anteriores no son necesarias, ya que el paquete trabaja con la enumeración de estados (`0` es el estado `en(l1)`, `1` es el estado `en(l2)`, ...) y de las acciones (`0` es la acción `esperar`, `1` es la acción `ir(l1, l2)`, ...). Pero nos servirán para transformar las respuestas de los algoritmos a un formato más amigable.

La acción `esperar` deja al robot en la misma localización en que se encuentre con probabilidad $1$ y tiene coste $0$. La función de transición del resto de acciones, sus costes y las recompensas de los estados se encuentran recogidas en el siguiente gráfico:

![](Recompensas_y_costes.png)

Las recompensas de los estados se representan mediante un array unidimensional.

In [216]:
#recompensas_estados = numpy.array([0, 0, 0, 100, -100])
recompensas_estados = numpy.array([-10000000, 0, 0, 0, -100, 100, 100, 100, 100]) 

print(recompensas_estados)

[-10000000         0         0         0      -100       100       100
       100       100]


La función de transición y el coste de aplicación para cada acción se representan, respectivamente, mediante un array bidimensional y un array unidimensional. Por ejemplo, para la acción `esperar` la función de transición vendrá dada por la matriz identidad $5 \times 5$ y el coste de aplicación por el vector nulo de longitud $5$.

In [217]:
'''
transición_esperar = numpy.array([[1, 0, 0, 0, 0],
                                  [0, 1, 0, 0, 0],
                                  [0, 0, 1, 0, 0],
                                  [0, 0, 0, 1, 0],
                                  [0, 0, 0, 0, 1]])
'''
transición_actua = numpy.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0.3, 0.7, 0, 0, 0, 0, 0, 0, 0],
                                [0.05, 0.25, 0.7, 0, 0, 0, 0, 0, 0],
                                [0, 0.1, 0.2, 0.7, 0, 0, 0, 0, 0],
                                [0, 0, 0.1, 0.2, 0.7, 0, 0, 0,0],
                                [0, 0, 0, 0, 0, 0.2, 0.8, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0.2, 0.8, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0.2, 0.8],
                                [0, 0, 0, 0, 0, 0, 0, 0, 1]])

#print(transición_esperar)
print(transición_actua)

[[1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.3  0.7  0.   0.   0.   0.   0.   0.   0.  ]
 [0.05 0.25 0.7  0.   0.   0.   0.   0.   0.  ]
 [0.   0.1  0.2  0.7  0.   0.   0.   0.   0.  ]
 [0.   0.   0.1  0.2  0.7  0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.2  0.8  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.2  0.8  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.2  0.8 ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [218]:
#coste_esperar = numpy.array([0, 0, 0, 0, 0])
#print(coste_esperar)
coste_actua = numpy.array([0, 0, 0, 0, numpy.inf, 0, 0, 0, 0])
print(coste_actua)

[ 0.  0.  0.  0. inf  0.  0.  0.  0.]


Un detalle importante es que el paquete asume que todas las acciones son ejecutables en todos los estados. Representaremos que una acción no es ejecutable en un estado estableciendo que el estado no cambia y que la ejecución tiene un coste muy alto (de hecho, podremos establecer coste infinito). De esta forma, el algoritmo no seleccionará la acción para la política óptima. Por ejemplo, para la acción `ir(l1, l2)` se tiene la siguiente matriz de transición y vector de costes:

In [219]:
'''

transición_ir_l1_l2 = numpy.array([[0, 1, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
'''
transición_viaja = numpy.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0.2, 0, 0, 0, 0, 0.8, 0, 0, 0],
                                [0.1, 0, 0, 0, 0, 0.3, 0.6, 0, 0],
                                [0,	0, 0, 0, 0, 0.1, 0.3, 0.6, 0],
                                [0, 0, 0, 0, 0, 0, 0.1, 0.3, 0.6],
                                [0.2, 0.8, 0, 0, 0, 0, 0, 0, 0],
                                [0.1, 0.3, 0.6, 0, 0, 0, 0, 0, 0],
                                [0,	0.1, 0.3, 0.6, 0, 0, 0, 0, 0],
                                [0, 0, 0.1, 0.3, 0.6, 0, 0, 0, 0]])
#print(transición_ir_l1_l2)
print(transición_viaja)

[[1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.8 0.  0.  0. ]
 [0.1 0.  0.  0.  0.  0.3 0.6 0.  0. ]
 [0.  0.  0.  0.  0.  0.1 0.3 0.6 0. ]
 [0.  0.  0.  0.  0.  0.  0.1 0.3 0.6]
 [0.2 0.8 0.  0.  0.  0.  0.  0.  0. ]
 [0.1 0.3 0.6 0.  0.  0.  0.  0.  0. ]
 [0.  0.1 0.3 0.6 0.  0.  0.  0.  0. ]
 [0.  0.  0.1 0.3 0.6 0.  0.  0.  0. ]]


In [220]:
#coste_ir_l1_l2 = numpy.array([100, numpy.inf, numpy.inf, numpy.inf, numpy.inf])
#print(coste_ir_l1_l2)

coste_viaja = numpy.array([numpy.inf, 0, 0, 0, 0, 0, 0, 0, numpy.inf])
print(coste_viaja)

[inf  0.  0.  0.  0.  0.  0.  0. inf]


Análogamente, para el resto de acciones se tienen las siguientes matrices de transición y vectores de coste:

In [221]:
transición_ir_l1_l4 = numpy.array([[.5, 0, 0, .5, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l1_l4 = numpy.array([1, numpy.inf, numpy.inf, numpy.inf, numpy.inf])

transición_ir_l2_l1 = numpy.array([[1, 0, 0, 0, 0],
                                   [1, 0, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l2_l1 = numpy.array([numpy.inf, 100, numpy.inf, numpy.inf, numpy.inf])

transición_ir_l2_l3 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 0, .8, 0, .2],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l2_l3 = numpy.array([numpy.inf, 1, numpy.inf, numpy.inf, numpy.inf])

transición_ir_l3_l2 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l3_l2 = numpy.array([numpy.inf, numpy.inf, 1, numpy.inf, numpy.inf])

transición_ir_l3_l4 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l3_l4 = numpy.array([numpy.inf, numpy.inf, 100, numpy.inf, numpy.inf])

transición_ir_l4_l1 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [1, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l4_l1 = numpy.array([numpy.inf, numpy.inf, numpy.inf, 1, numpy.inf])

transición_ir_l4_l3 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 0, 1]])
coste_ir_l4_l3 = numpy.array([numpy.inf, numpy.inf, numpy.inf, 100, numpy.inf])

transición_ir_l4_l5 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 0, 1],
                                   [0, 0, 0, 0, 1]])
coste_ir_l4_l5 = numpy.array([numpy.inf, numpy.inf, numpy.inf, 100, numpy.inf])

transición_ir_l5_l2 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 1, 0, 0, 0]])
coste_ir_l5_l2 = numpy.array([numpy.inf, numpy.inf, numpy.inf, numpy.inf, 1])

transición_ir_l5_l4 = numpy.array([[1, 0, 0, 0, 0],
                                   [0, 1, 0, 0, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 1, 0],
                                   [0, 0, 0, 1, 0]])
coste_ir_l5_l4 = numpy.array([numpy.inf, numpy.inf, numpy.inf, numpy.inf, 100])

Finalmente, para calcular una política óptima del proceso de decisión de Markov hay que pasarle a los algoritmos un array tridimensional de todas las matrices de transición y un array bidimensional de las recompensas de los estados menos los costes de las acciones.

In [222]:
'''
transiciones_sistema = numpy.array([transición_esperar,
                                    transición_ir_l1_l2,
                                    transición_ir_l1_l4,
                                    transición_ir_l2_l1,
                                    transición_ir_l2_l3,
                                    transición_ir_l3_l2,
                                    transición_ir_l3_l4,
                                    transición_ir_l4_l1,
                                    transición_ir_l4_l3,
                                    transición_ir_l4_l5,
                                    transición_ir_l5_l2,
                                    transición_ir_l5_l4])
'''

transiciones_sistema = numpy.array([transición_actua,
                                    transición_viaja])

print(transiciones_sistema)

[[[1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.3  0.7  0.   0.   0.   0.   0.   0.   0.  ]
  [0.05 0.25 0.7  0.   0.   0.   0.   0.   0.  ]
  [0.   0.1  0.2  0.7  0.   0.   0.   0.   0.  ]
  [0.   0.   0.1  0.2  0.7  0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.2  0.8  0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.2  0.8  0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.2  0.8 ]
  [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]

 [[1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.2  0.   0.   0.   0.   0.8  0.   0.   0.  ]
  [0.1  0.   0.   0.   0.   0.3  0.6  0.   0.  ]
  [0.   0.   0.   0.   0.   0.1  0.3  0.6  0.  ]
  [0.   0.   0.   0.   0.   0.   0.1  0.3  0.6 ]
  [0.2  0.8  0.   0.   0.   0.   0.   0.   0.  ]
  [0.1  0.3  0.6  0.   0.   0.   0.   0.   0.  ]
  [0.   0.1  0.3  0.6  0.   0.   0.   0.   0.  ]
  [0.   0.   0.1  0.3  0.6  0.   0.   0.   0.  ]]]


In [223]:
# Transformamos el vector de recompensas en una matriz 5x1

#matriz_recompensas = recompensas_estados.reshape(5, 1)
matriz_recompensas = recompensas_estados.reshape(9, 1)

# Creamos una matriz donde cada columna es el vector de costes de una acción
'''
matriz_costes = numpy.column_stack([coste_esperar,
                                    coste_ir_l1_l2,
                                    coste_ir_l1_l4,
                                    coste_ir_l2_l1,
                                    coste_ir_l2_l3,
                                    coste_ir_l3_l2,
                                    coste_ir_l3_l4,
                                    coste_ir_l4_l1,
                                    coste_ir_l4_l3,
                                    coste_ir_l4_l5,
                                    coste_ir_l5_l2,
                                    coste_ir_l5_l4])
'''
matriz_costes = numpy.column_stack([coste_actua,
                                    coste_viaja])

recompensas_sistema = matriz_recompensas - matriz_costes
print(recompensas_sistema)

[[-1.e+07    -inf]
 [ 0.e+00  0.e+00]
 [ 0.e+00  0.e+00]
 [ 0.e+00  0.e+00]
 [   -inf -1.e+02]
 [ 1.e+02  1.e+02]
 [ 1.e+02  1.e+02]
 [ 1.e+02  1.e+02]
 [ 1.e+02    -inf]]


Para obtener una política óptima mediante el algoritmo de iteración de valores hay que crear una instancia adecuada de la clase `ValueIteration`. Hay que tener en cuenta que el valor de $\epsilon$ que se proporciona es la diferencia máxima que se requiere entre el último $U_{n}$ calculado y $U^{*}$, para lo que se deriva a partir de él la diferencia máxima adecuada entre $U_{n}$ y $U_{n - 1}$. Por otra parte, $U_{0}$ asigna utilidad inicial $0$ a todos los estados.

In [224]:
ejemplo_robot_VI = mdp.ValueIteration(
    transitions=transiciones_sistema,
    reward=recompensas_sistema,
    discount=0.9,
    epsilon=0.1
)

Para ejecutar el algoritmo basta usar el método `run` de la clase anterior, pero antes establecemos el modo verboso, para que proporcione detalles de los cálculos.

In [225]:
ejemplo_robot_VI.setVerbose()
ejemplo_robot_VI.run()

  Iteration		V-variation
    1		  10000100.0
    2		  9000090.0
    3		  8100081.0
    4		  7290072.9
    5		  6561065.61
    6		  5904959.049
    7		  5314463.1441
    8		  4783016.82969
    9		  4304715.146721002
    10		  3874243.6320489007
    11		  3486819.2688440103
    12		  3138137.3419596176
    13		  2824323.6077636383
    14		  2541891.24698728
    15		  2287702.122288564
    16		  2058931.9100596942
    17		  1853038.7190537367
    18		  1667734.8471483528
    19		  1500961.362433519
    20		  1350865.2261901656
    21		  1215778.7035711533
    22		  1094200.8332140455
    23		  984780.7498926291
    24		  886302.6749033766
    25		  797672.407413036
    26		  717905.1666717264
    27		  646114.6500045627
    28		  581503.1850041035
    29		  523352.86650369014
    30		  471017.5798533285
    31		  423915.8218679957
    32		  381524.2396811961
    33		  343371.81571307057
    34		  309034.634141765
    35		  278131.17072759
    36		  250318.05365482502
    37		  225286.2482

La política óptima se encuentra guardada en el atributo `policy`.

In [226]:
ejemplo_robot_VI.policy

(0, 1, 1, 1, 1, 0, 0, 0, 0)

Con la siguiente expresión podemos verla de manera más amigable.

In [227]:
for estado, i in zip(estados, ejemplo_robot_VI.policy):
    print(f'En el estado {estado} ejecuta la acción {acciones[i]}')

En el estado M ejecuta la acción actua
En el estado PC2 ejecuta la acción viaja
En el estado PC3 ejecuta la acción viaja
En el estado PC4 ejecuta la acción viaja
En el estado PC5 ejecuta la acción viaja
En el estado A2 ejecuta la acción actua
En el estado A3 ejecuta la acción actua
En el estado A4 ejecuta la acción actua
En el estado A5 ejecuta la acción actua


De manera análoga, se puede obtener una política óptima mediante el algoritmo de iteración de políticas creando una instancia adecuada de la clase `PolicyIteration`.

In [228]:
ejemplo_robot_PI = mdp.PolicyIteration(
    transitions=transiciones_sistema,
    reward=recompensas_sistema,
    discount=0.9,
    policy0=numpy.array([0, 0, 0, 4, 0, 0, 5, 0, 0])  # La política inicial es esperar en cada estado
)
ejemplo_robot_PI.setVerbose()
print(ejemplo_robot_PI)

ejemplo_robot_PI.run()

print()
for estado, i in zip(estados, ejemplo_robot_PI.policy):
    print(f'En el estado {estado} ejecuta la acción {acciones[i]}')

P: 
array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.3 , 0.7 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.05, 0.25, 0.7 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.1 , 0.2 , 0.7 , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.1 , 0.2 , 0.7 , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.8 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])
array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.2, 0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0. ],
       [0.1, 0. , 0. , 0. , 0. , 0.3, 0.6, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.1, 0.3, 0.6, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0.3, 0.6],
       [0.2, 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0.3, 0.6, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.1, 0.3, 0.6, 0. , 0. , 0. , 0. , 0.

# Ejemplo de la piscifactoría

Estamos a cargo de una piscifactoría. Cada temporada hay que decidir qué parte de la población de peces se captura y comercializa y qué parte se deja en la piscifactoría para que se reproduzcan. Si denotamos $x$ la cantidad de peces que hay en la piscifactoría al inicio de la temporada, la recompensa que se obtiene al comercializar $y < x$ peces es $10y$. Si $z = x - y$ es la cantidad de peces que quedan, entonces para la siguiente temporada puede haber ocurrido uno de los siguientes casos:
* Con probabilidad $0.2$, la temporada ha sido buena y los peces se han reproducido hasta alcanzar la cantidad de $1.8z$.
* Con probabilidad $0.7$, la temporada ha sido normal y los peces se han reproducido hasta alcanzar la cantidad de $1.4z$.
* Con probabilidad $0.1$, la temporada ha sido mala y los peces no se han reproducido lo suficiente, por lo que queda una cantidad de $0.9z$.

En todos los casos se redondea hacia arriba y hay que tener también en cuenta que la piscifactoría tiene una capacidad máxima de $N$ peces. También puede ocurrir que coincidan alguno, o incluso todos, los valores anteriores, en cuyo caso las probabilidades correspondientes se suman.

Por ejemplo, supongamos que $N = 10$. Entonces:
* Los posibles estados son: $0, 1, \dotsc, 10$.
* La recompensa de cada estado es $0$, ya que lo que se pretende es maximizar los beneficios de comercializar los peces, más que disponer de una cierta cantidad de peces en la piscifactoría (una variante a considerar sería establecer recompensas menores para los estados con menos peces, para así evitar las políticas que dejen pocos peces en la piscifactoría).
* Las posibles acciones son comercializar $y$ peces, para cada $y = 0, 1, \dotsc, 10$.
* El coste de comercializar $y$ peces es $-10y$. Obsérvese que son valores negativos para considerarlos como beneficio de aplicar la acción.
* Sin embargo, si en la piscifactoría hay $x$ peces, entonces el coste de comercializar $y > 0$ peces, con $y \geq x$, es $\infty$, ya que esas acciones no serían aplicables. Comercializar $0$ peces siempre es aplicable.
* Si en la piscifactoría hay $x = 8$ peces y comercializamos $y = 2$ peces, entonces la función de transición para esa acción sería:
  * Con probabilidad 0.2 pasaríamos al estado $1.8 (8 - 2) = 10$ (ya que esa es la capacidad máxima de la factoría).
  * Con probabilidad 0.7 pasaríamos al estado $1.4 (8 - 2) = 9$.
  * Con probabilidad 0.1 pasaríamos al estado $0.9 (8 - 2) = 6$.

**Ejercicio 1**: Definir una función que, dadas la capacidad $N$ de la piscifactoria y la cantidad $y$ de peces a comercializar, devuelva el vector de costes de esa acción.

**Ejercicio 2**: Definir una función que, dadas la capacidad $N$ de la piscifactoria y la cantidad $y$ de peces a comercializar, devuelva la matriz de transición de esa acción.

**Ejercicio 3**: Definir una función que, dada la capacidad $N$ de la piscifactoría, devuelva la matriz de recompensas del proceso de decisión de Markov correspondiente.

**Ejercicio 4**: Definir una función que, dada la capacidad $N$ de la piscifactoría, devuelva el array de matrices de transición del proceso de decisión de Markov correspondiente.

**Ejercicio 5**: Aplicar los algoritmos de iteración de valores y políticas para calcular políticas óptimas considerando distintos valores de $N$ (por ejemplo, distintas potencias de $10$) y del factor de descuento (por ejemplo, $\gamma = 0.1, 0.5, 0.9$, para tener poco, algo o mucho en cuenta lo que pueda ocurrir en el futuro).